In [1]:
import sys
!{sys.executable} -m pip install -r requirements.txt
import pydicom
import matplotlib.pylab as plt
import matplotlib
import os
import numpy as np
from PIL import Image
import time


You should consider upgrading via the '/opt/python/latest/bin/python -m pip install --upgrade pip' command.


Configure Global Variables
----

In [2]:
window = 350 # Window/Level from Hoori et al ("HU-attention-window with a window/level of 350/40 HU")
level = 40
bitDepth = 8 # Must be 8 for uint8 PNGs. Explore floating point
TrainingDataRootDir = '../Training Data'
TestDataRootDir = '../Test Data'

Preprocess Training Data
----

In [3]:
CombinedTrainingImageDir = TrainingDataRootDir + '/AllImages'
CombinedTrainingMaskDir = TrainingDataRootDir + '/AllMasks'
os.makedirs(CombinedTrainingImageDir, exist_ok=True)
os.makedirs(CombinedTrainingMaskDir, exist_ok=True)

currentImageIndex = 0
subjects = os.listdir(TrainingDataRootDir)
for subject in subjects:
    if not (subject == 'AllImages' or subject == 'AllMasks'):
        currentSubjectDir = TrainingDataRootDir + "/" + subject;
        print(currentSubjectDir)

        volume_dicoms = os.listdir(currentSubjectDir + '/Dicom/Vol/')
        dataset_vol = [None] * (len(volume_dicoms))
        for dicom in volume_dicoms:
            slice_number = int(dicom.split('Slice')[1].split('.dcm')[0])
            dataset_vol[slice_number-1] = pydicom.dcmread(currentSubjectDir+'/Dicom/Vol/'+dicom)
        
        sac_dicoms = os.listdir(currentSubjectDir + '/Dicom/Sac/')
        dataset_sac = [None] * (len(sac_dicoms))
        for dicom in sac_dicoms:
            slice_number = int(dicom.split('Slice')[1].split('.dcm')[0])
            dataset_sac[slice_number-1] = pydicom.dcmread(currentSubjectDir+'/Dicom/Sac/'+dicom)

        for i in range(1, len(dataset_vol)-1):
            sliceRGB = np.zeros((dataset_vol[i].pixel_array.shape[0],dataset_vol[i].pixel_array.shape[1],3))
            sliceRGB[:,:,0] = dataset_vol[i-1].pixel_array
            sliceRGB[:,:,1] = dataset_vol[i].pixel_array
            sliceRGB[:,:,2] = dataset_vol[i+1].pixel_array
            sliceRGB = np.clip((sliceRGB - (level-window/2)) / window, 0,1) # Apply window/level and clip to 0->1 range
            sliceRGB = sliceRGB * ((2**bitDepth)-1) # Scale to bitdepth
            sliceRGB = np.floor(sliceRGB) # Floor to integer
            imageFilename = CombinedTrainingImageDir + "/" + str(currentImageIndex)+'.png'
            matplotlib.image.imsave(imageFilename, sliceRGB.astype(np.uint8))

            sliceMask = np.zeros((dataset_vol[i].pixel_array.shape[0],dataset_vol[i].pixel_array.shape[1],3))
            sliceMask[:,:,0] = dataset_sac[i].pixel_array
            sliceMask[:,:,1] = dataset_sac[i].pixel_array
            sliceMask[:,:,2] = dataset_sac[i].pixel_array
            sliceMask = sliceMask * ((2**bitDepth)-1) # Scale to bitdepth
            maskFilename = CombinedTrainingMaskDir + "/" + str(currentImageIndex)+'.png'
            matplotlib.image.imsave(maskFilename, sliceMask.astype(np.uint8))

            currentImageIndex = currentImageIndex + 1

../Training Data/CA003
../Training Data/CA002


Preprocess Test Data
----

In [ ]:
CombinedTestImageDir = TestDataRootDir + '/AllImages'
CombinedTestMaskDir = TestDataRootDir + '/AllMasks'
os.makedirs(CombinedTestImageDir, exist_ok=True)
os.makedirs(CombinedTestMaskDir, exist_ok=True)

currentImageIndex = 0
subjects = os.listdir(TestDataRootDir)
for subject in subjects:
    if not (subject == 'AllImages' or subject == 'AllMasks'):
        currentSubjectDir = TestDataRootDir + "/" + subject;
        print(currentSubjectDir)

        volume_dicoms = os.listdir(currentSubjectDir + '/Dicom/Vol/')
        dataset_vol = [None] * (len(volume_dicoms))
        for dicom in volume_dicoms:
            slice_number = int(dicom.split('Slice')[1].split('.dcm')[0])
            dataset_vol[slice_number-1] = pydicom.dcmread(currentSubjectDir+'/Dicom/Vol/'+dicom)
        
        sac_dicoms = os.listdir(currentSubjectDir + '/Dicom/Sac/')
        dataset_sac = [None] * (len(sac_dicoms))
        for dicom in sac_dicoms:
            slice_number = int(dicom.split('Slice')[1].split('.dcm')[0])
            dataset_sac[slice_number-1] = pydicom.dcmread(currentSubjectDir+'/Dicom/Sac/'+dicom)

        for i in range(1, len(dataset_vol)-1):
            sliceRGB = np.zeros((dataset_vol[i].pixel_array.shape[0],dataset_vol[i].pixel_array.shape[1],3))
            sliceRGB[:,:,0] = dataset_vol[i-1].pixel_array
            sliceRGB[:,:,1] = dataset_vol[i].pixel_array
            sliceRGB[:,:,2] = dataset_vol[i+1].pixel_array
            sliceRGB = np.clip((sliceRGB - (level-window/2)) / window, 0,1) # Apply window/level and clip to 0->1 range
            sliceRGB = sliceRGB * ((2**bitDepth)-1) # Scale to bitdepth
            sliceRGB = np.floor(sliceRGB) # Floor to integer
            imageFilename = CombinedTestImageDir + "/" + str(currentImageIndex)+'.png'
            matplotlib.image.imsave(imageFilename, sliceRGB.astype(np.uint8))

            sliceMask = np.zeros((dataset_vol[i].pixel_array.shape[0],dataset_vol[i].pixel_array.shape[1],3))
            sliceMask[:,:,0] = dataset_sac[i].pixel_array
            sliceMask[:,:,1] = dataset_sac[i].pixel_array
            sliceMask[:,:,2] = dataset_sac[i].pixel_array
            sliceMask = sliceMask * ((2**bitDepth)-1) # Scale to bitdepth
            maskFilename = CombinedTestMaskDir + "/" + str(currentImageIndex)+'.png'
            matplotlib.image.imsave(maskFilename, sliceMask.astype(np.uint8))

            currentImageIndex = currentImageIndex + 1